In [4]:
# "predict median for that building & meter, for that day of week"

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv("../input/ashrae-energy-prediction/train.csv")


In [5]:
# add day of week and day of year

df["timestamp"] = pd.to_datetime(df["timestamp"])
df['dayofweek'] = df["timestamp"].dt.dayofweek
df['dayofyear'] = df["timestamp"].dt.dayofyear

df.sample(n=10, random_state=42)

,building_id,meter,timestamp,meter_reading,dayofweek,dayofyear
14245562,1324,1,2016-09-16 16:00:00,0.0000,4,260
1282718,1013,0,2016-01-24 06:00:00,32.0001,6,24
13883790,229,1,2016-09-10 07:00:00,567.6550,5,254
4781820,217,3,2016-04-01 01:00:00,0.0000,4,92
10415393,1434,0,2016-07-10 04:00:00,65.7500,6,192
1057008,1047,0,2016-01-20 04:00:00,90.9833,2,20
4507399,911,1,2016-03-26 20:00:00,295.0640,5,86
19478829,1039,0,2016-12-18 23:00:00,16.9000,6,353
8955615,265,0,2016-06-14 06:00:00,128.3700,1,166
13799839,896,0,2016-09-08 19:00:00,300.0000,3,252


In [15]:
# sum each day for meters 
# (note we might want to look at imputing missing values)

df_sum = df.groupby(['building_id','meter','dayofyear','dayofweek',]).sum()
df_sum.reset_index(inplace=True)
df_sum.drop(['dayofyear'], axis=1, inplace=True)
df_sum.loc[df_sum['building_id'] == 1047].sort_values(['meter','dayofweek',])

,building_id,meter,dayofweek,meter_reading
543115,1047,0,0,4581.4444
543122,1047,0,0,4230.9262
543129,1047,0,0,4766.6572
543136,1047,0,0,4948.4498
543143,1047,0,0,2453.8984
543150,1047,0,0,3590.6167
543157,1047,0,0,4961.3930
543164,1047,0,0,4971.8259
543171,1047,0,0,5009.6553
543178,1047,0,0,1643.2475


In [18]:
one_hot = df_sum.get_dummies(df_sum['building_id','meter','dayofweek'])

AttributeError: 'DataFrame' object has no attribute 'get_dummies'

In [16]:
# Split fit and predit
X_train, X_test, y_train, y_test = train_test_split(
    df_sum.drop('meter_reading', axis=1), df_sum['meter_reading'], 
    test_size=0.50, random_state=42)

dummy_mean = DummyRegressor(strategy='median')

reg = dummy_mean.fit(X_train, y_train)
predict_median = reg.predict(X_test)
print(predict_median)

# calcuate error, which looks really high
mean_absolute_error(y_test, predict_median)

[2045.44305 2045.44305 2045.44305 ... 2045.44305 2045.44305 2045.44305]


49203.03024262267